Есть парикмахерская, с n парикмахерами и очередью. Предполагается, что поток клиентов,
желающих постричься – пуассоновский с интенсивностью λ, поток постриженных i-м
парикмахером клиентов – пуассоновский с интенсивностью μi
.
1. Выбрав значения n, λ и μi
, написать программу, моделирующую работу этой
парикмахерской в течение рабочего дня с 10:00 до 20:00. Результатом работы программы
может быть текстовый файл, каждая строка которого отражает событие в работе
парикмахерской: появление нового клиента или окончание стрижки клиента. Файл может
выглядеть примерно так (нули и единицы перед слешем показывают, какой из
парикмахеров занят, а какой свободен; число после слеша показывает число клиентов в
очереди):
10:03 - 0 0 1 / 0
10:11 - 1 0 1 / 0
10:18 - 1 1 1 / 0
10:24 - 1 1 0 / 0
10:27 - 1 1 1 / 0
10:34 - 1 1 1 / 1
10:39 - 1 1 1 / 2
10:42 - 1 1 1 / 1
….
Более продвинутые интерфейсы, например графические, изображающие работу системы в
режиме реального времени, приветствуются, но не обязательны.
2. Подобрать значения n, λ и μi так, чтобы результаты работы парикмахерской выглядели
правдоподобно, т. е. чтобы не образовывалась гигантская очередь, стрижка не занимала
бы 1-2 минуты и т.п.
3. Написать словами свои соображения о том, как должны соотноситься друг с другом
интенсивность потока клиентов и суммарная интенсивность работы парикмахеров, чтобы
парикмахерская работала оптимальным образом (с одной стороны без многолюдных
очередей, с другой – без постоянного простоя парикмахеров).

In [42]:
import numpy as np 
import pandas as pd 
from numpy import cumsum

In [52]:
workers_n = 4 #задаем количество парикмахеров
worktime = [8,22] #задаем время работы парикмахерской, тут с 8 до 22
entensive = 15 #интенсивность потока клиентов мин

def init_params(workers_n,worktime,entensive ):
    workers_effectivity = []
    for i in range(workers_n):
        workers_effectivity.append(abs(np.random.normal(6,1)*10))
    print('Эффективность работников:', workers_effectivity) #сколько в среднем каждый тратит времени на стрижку
    workers_work_done = []
    for i in range(workers_n):
        workers_work_done.append(np.random.poisson(workers_effectivity[i],100))  
    print('Ряд событий затрат времени на стрижку для первого работника:',workers_work_done[0]) 
    work_to_do = np.random.poisson(entensive,200) #поступление заказов
    print('Ряд поступления новых заказов:', work_to_do)
    return workers_effectivity, workers_work_done, work_to_do
    
workers_effectivity, workers_work_done, work_to_do = init_params(workers_n,worktime,entensive )
    

Эффективность работников: [62.417592201625915, 68.14937845788991, 74.07473061498754, 71.57083431392543]
Ряд событий затрат времени на стрижку для первого работника: [72 70 69 58 42 68 54 60 56 64 55 57 66 65 65 65 70 68 50 72 64 62 49 50
 53 59 62 62 72 54 64 51 55 61 59 57 66 62 71 72 67 48 55 66 50 61 69 55
 72 52 61 52 61 64 81 72 63 71 83 62 71 57 72 66 63 61 62 60 80 62 50 64
 58 61 55 54 65 64 60 73 62 71 73 61 67 61 72 69 73 58 70 60 63 75 50 58
 72 68 63 64]
Ряд поступления новых заказов: [11 11 11 14 18 18 14  8 13 15 13  9 10 16 18 10 14 11 13 14 22 14 17 12
 21  4 16 19 16 16 16 18 12 18 17 17 20 15  9 23 11 13 21 21 12 10 16 15
 17 15 17 16 16  6 18 18 17 11  9  9 14 13 16 15 21 18 14 14 13  9 13  9
 13 14 16 17 18 11 17 21 14 12 20 14 16  8 15 14 17 12 13 15 19 11 12 10
 15 14 17 16 19 16 16  9 23 13 19 20 15 16 15 11 11 16 23 14 20 16 13 18
 19 23 14 20 12 12 15 19 21 13 19 18 15 17 10 23  9 15 12 19 10 13 22 16
 17 18  9 15 18 22 21 13  9 14 17 18 19 16 19 16 14 23 11 16

In [45]:
minutes_worktime = (worktime[1]  - worktime[0])*60
def convert_minute_into_time(worktime, current_minute): #функция возвращает время из номера минуты
    hours = int(current_minute/60) + worktime[0] 
    minutes = current_minute%60
    return str(hours)+" : "+str(minutes)

print(convert_minute_into_time(worktime, 167))

10 : 47


In [53]:
def upate_current_worktime(current, minute,workers_work_done,minutes_works):
    for key in current:
        if (current[key] != 0)&(key!='quote'):
            current[key] = current[key]-1
            
    if minute in minutes_works:
        current['quote'] +=1
                
    while current['quote'] > 0:
        current_q= current['quote']
        for key in current:
            if (current[key] == 0)&(key!='quote'):
                current[key] = workers_work_done[key][0]
                workers_work_done[key] = workers_work_done[key][1:]
                current['quote'] -= 1
                break
        if current['quote'] == current_q:
            break
                
    return current, workers_work_done
                
def work_imitation(workers_n, worktime,entensive, workers_work_done, work_to_do,minutes_worktime):
    minutes_works = cumsum(work_to_do) #на какой минуте поступят новые заказы
    current = {i:0 for i in range(workers_n) }
    current['quote'] = 0
    print('Начальное состояние: ',current)
    
    for minute in range(minutes_worktime):
        current, workers_work_done = upate_current_worktime(current, minute,workers_work_done,minutes_works)
        print(convert_minute_into_time(worktime, minute),current)
        
        

In [55]:
workers_n = 4 #задаем количество парикмахеров
worktime = [8,22] #задаем время работы парикмахерской, тут с 8 до 22
entensive = 15 #интенсивность потока клиентов мин

workers_effectivity, workers_work_done, work_to_do = init_params(workers_n,worktime,entensive )
work_imitation(workers_n, worktime,entensive, workers_work_done, work_to_do,minutes_worktime)

#время, номер парикмахера : сколько минут ему еще работать над заказом (если 0 то он свободен), 'quote': размер очереди

Эффективность работников: [45.08315769747347, 53.42000570219107, 69.21098968565877, 58.639266819489194]
Ряд событий затрат времени на стрижку для первого работника: [46 36 28 44 41 58 52 36 43 50 43 41 41 48 38 45 41 63 67 40 51 41 45 46
 44 45 33 46 46 52 53 33 40 54 44 41 45 47 40 45 44 28 54 43 37 52 41 51
 45 44 53 52 40 39 43 56 49 41 50 37 56 51 38 47 45 41 46 52 51 55 41 48
 41 45 50 46 34 37 30 44 46 51 44 37 50 43 51 40 38 47 54 59 47 48 37 44
 41 43 33 55]
Ряд поступления новых заказов: [13 19 17 11 19 14 18 16 11 15  8 16 14 13 22 12 21 18 18 20 19 12 12 10
 14 21 13 16 18  8 11  9 21 21 16 12 23 19 16 20 14 19 10  8 13  8 11 11
 14 14 14  7 13 14 12 17 15 11 13 20 15 13 13 19 14 18 13 16 11 16 17 15
 19 14 10 18 18 21 16 14 13 17 14 16 11  9 16 10 11 18 13 15 19 16 20 10
 11 13 14 11 15 11 18 20 14 13 14 17 13  8  9 12 14 19 16 15 10 14 17 14
 11 12 17 19 15 13 10 16 25 19 15 19 17 15 19 14 13  9 24 18 19  9 19 13
 17 14 17 19 17 12 15 24 15 10 15 12 17 13 19 11 10 13 14 14

Если ставить суммарную интенсивность обработки заявок примерно равной интенсивности потока, то очередь под конец дня как правило небольшая или ее совсем нет. (зависит от того какая эффективность парикмахеров на смене)

In [56]:
workers_n = 5 #задаем количество парикмахеров
worktime = [8,22] #задаем время работы парикмахерской, тут с 8 до 22
entensive = 10 #интенсивность потока клиентов мин

workers_effectivity, workers_work_done, work_to_do = init_params(workers_n,worktime,entensive )
work_imitation(workers_n, worktime,entensive, workers_work_done, work_to_do,minutes_worktime)

#тут в среднем в час паркимахерская может обрабатывать ~5 клиентов, а их приходит 6 и поэтому к концу дня образуется очередь

Эффективность работников: [63.2871430018408, 56.315959832646946, 57.677695740031865, 49.97708794837134, 65.39640995571614]
Ряд событий затрат времени на стрижку для первого работника: [76 72 56 63 47 62 59 68 79 71 59 52 76 76 69 69 50 63 59 57 63 59 54 55
 62 57 53 59 72 73 73 65 71 67 66 54 72 57 71 66 53 66 74 58 60 62 63 59
 66 60 65 63 65 64 51 75 63 65 63 66 76 78 53 60 67 69 61 72 61 55 62 67
 65 74 66 55 76 73 57 69 78 77 53 75 52 63 58 56 57 62 77 55 65 77 63 61
 54 63 55 61]
Ряд поступления новых заказов: [ 6  9 13 11  7 10 10 10  9  8  9  8  9  6  8  3 11  7  4 11  7 12  5 11
  9  2  9 11  5 16  9 13  9  7  8 10  6  6 15  5 12  8  9 12  9 10 13  9
 10 12 12  9 18  9  6 11 11 10  9  7  7  9  5  7 12  7  7  6 13 16  5 14
 10  6 11 12 12 12  9 14 12 16 12  6 10 10 14 12 12  9 11 12 13 19  6  7
  1  8  7  5  9  7 15  5  9  8 16  7  9  9 13  8 11  9 10 11  8 14  6 10
 10 11 13 13  9 15  9  7 10 11 10  8  6  7  8  8  8 17 10  7  9 10 11  9
 12 10 10 13 13 14  7  6  6  8 10 13  8  

In [57]:
workers_n = 3 #задаем количество парикмахеров
worktime = [8,22] #задаем время работы парикмахерской, тут с 8 до 22
entensive = 30 #интенсивность потока клиентов мин

workers_effectivity, workers_work_done, work_to_do = init_params(workers_n,worktime,entensive )
work_imitation(workers_n, worktime,entensive, workers_work_done, work_to_do,minutes_worktime)

#тут в среднем 3 паркимахера работаю на 2 заказами в час, поэтому очереди почти не бывает


Эффективность работников: [64.04425498153729, 69.66715698542116, 65.87917060917269]
Ряд событий затрат времени на стрижку для первого работника: [74 53 47 58 60 63 55 54 59 67 59 60 73 64 51 69 64 70 66 69 71 63 59 71
 58 54 72 68 65 68 54 67 65 61 60 61 74 56 85 57 74 68 66 63 57 67 85 67
 66 67 73 69 76 70 64 66 67 74 73 59 81 66 47 66 67 56 71 59 88 54 59 70
 81 60 67 74 56 64 60 49 60 67 78 66 65 71 62 64 63 80 61 57 72 64 61 79
 70 50 64 54]
Ряд поступления новых заказов: [32 28 22 35 40 32 25 33 32 32 35 37 31 23 31 35 34 28 33 23 33 30 29 27
 27 40 32 33 27 33 24 29 29 44 40 32 25 30 24 30 32 39 27 39 29 33 30 32
 32 23 27 35 42 34 22 28 32 37 43 37 17 27 22 45 32 35 32 26 28 26 24 31
 32 30 40 23 22 33 39 41 22 29 33 37 25 38 43 20 31 29 27 21 36 19 21 27
 30 39 26 32 28 27 28 23 25 30 27 27 38 28 25 29 27 29 27 31 26 17 32 35
 41 22 31 35 35 27 27 30 29 26 38 35 31 33 35 26 34 31 31 42 30 30 31 23
 34 20 30 29 38 29 44 40 25 23 25 33 33 48 28 30 29 28 16 33 41 22 29 29
 31 31 